In [1]:
import sympy as sym
import numpy as np
import time
import matplotlib.pyplot as plt
import itertools as it

In [2]:
# Set font size of plot elements\n",
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 18
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## Functions

In [3]:
def generate_symbols(base, indices):
    """
    Function for generating symbols to be used in expressions
    :param base:    the string to use as the base of the symbol
    :param indices: a list of indices to append to the base
    :return:        the list of symbols
    """
    syms = [sym.symbols(base + "_" + str(indices[i])) for i in range(len(indices))]
    return syms

In [4]:
def generate_H(dim, freq_symbols, rabi_symbols):
    """
    Function for generating the Hamiltonian matrix from lists of symbols
    :param dim:            the number of rows/cols the matrix should have
    :param freq_symbols:   the list of frequency symbols i.e. the diagonal elements
    :param rabi_symbols:   the list of rabi symbols i.e. the off-diagonal elements
    :return:               the Hamiltonian matrix
    """
    # Define the basis vectors
    basis = [sym.Array(np.eye(dim))[i] for i in range(dim)]

    # Create the blank Hamiltonian
    H = sym.Matrix([[0 for i in range(dim)] for j in range(dim)])

    # Add the diagonal elements
    for i in range(dim):
        H += freq_symbols[i] * sym.tensorproduct(basis[i], basis[i]).reshape(dim, dim).tomatrix()

    # Add the off-diagonal elements
    for i in range(dim - 1):
        H += rabi_symbols[0][i] * sym.tensorproduct(basis[i], basis[i+1]).reshape(dim, dim).tomatrix()
        H += rabi_symbols[1][i] * sym.tensorproduct(basis[i+1], basis[i]).reshape(dim, dim).tomatrix()

    return H
    

In [5]:
def generate_ρ(dim):
    """
    Function for generating the density matrix
    :param dim:  the number of rows/cols the matrix should have
    """
    # Define the basis vectors
    basis = [sym.Array(np.eye(dim))[i] for i in range(dim)]

    # Define the symbols
    ρ_syms = [[sym.symbols("ρ_" + str(i+1) + "_" + str(j+1)) for j in range(dim)] for i in range(dim)]

    # Set the trace preserving condition
    ρ_syms[dim-1][dim-1] = 1 - np.sum([ρ_syms[i][i] for i in range(dim-1)])

    # Define the shape of the density matrix
    ρ = sym.Matrix([[0 for i in range(dim)] for j in range(dim)])

    # Fill in
    for i in range(dim):
        for j in range(dim):
            ρ += ρ_syms[i][j] * sym.tensorproduct(basis[i], basis[j]).reshape(dim, dim).tomatrix()

    return ρ

In [6]:
def generate_jump_ops(dim):
    """
    Function for generating the jump operators
    """
    # Define the basis vectors
    basis = [sym.Array(np.eye(dim))[i] for i in range(dim)]

    # Define the jump operators
    σ = []
    for i in range(dim-1):
        σ.append(sym.tensorproduct(basis[i], basis[i+1]).reshape(dim, dim).tomatrix())

    return σ

In [7]:
def generate_Liouvillian(ρ, H, jump_ops, γs):
    """
    Function for generating the Liouvillian i.e. the rhs of the Lindblad master equation
    :param ρ:         the density matrix
    :param H:         the system Hamiltonian
    :param jump_ops:  the transition operators of the system
    :param γs:        the decay rates associated with the jump operators
    :return:          the Liouvillian
    """
    # Calculate the commutator first
    comm = H @ ρ - ρ @ H

    # Now calculate the Lindbladian term
    lindblad_term = sym.Matrix([[0 for i in range(ρ.shape[0])] for j in range(ρ.shape[0])])
    for i in range(len(jump_ops)):
        lindblad_term += γs[i] * jump_ops[i] @ ρ @ jump_ops[i].H
        lindblad_term -= γs[i] / 2 * (jump_ops[i].H @ jump_ops[i] @ ρ + ρ @ jump_ops[i].H @ jump_ops[i])

    return -1j * comm + lindblad_term

In [8]:
def generate_vector_Liouvillian(H, jump_ops, γs):
    """
    Function for generating the vectorized version of the Liouvillian to be applied to the density matrix
    """
    # Define the identity matrix for ease of use
    id = sym.eye(H.shape[0])

    # First do the commutator part
    comm = sym.tensorproduct(H, id) - sym.tensorproduct(id, H.T)

    # Now do the lindblad part
    lindblad_term = sym.Array([[[[0 for i in range(H.shape[0])] for j in range(H.shape[0])]
                                for k in range(H.shape[0])] for l in range(H.shape[0])])
    for i in range(len(jump_ops)):
        lindblad_term += γs[i] * sym.tensorproduct(jump_ops[i], jump_ops[i].conjugate())
        lindblad_term -= γs[i] / 2 * sym.tensorproduct(jump_ops[i].H @ jump_ops[i], id)
        lindblad_term -= γs[i] / 2 * sym.tensorproduct(id, (jump_ops[i].H @ jump_ops[i]).T)

    # Combine
    L = -1j * comm + lindblad_term
    
    # Using reshape to get this to a 4x4 doesn't work like we want, so we have to do this manually
    # First, permute the first two axes
    L = sym.permutedims(L, (1, 0, 2, 3))

    # Then loop over the 4 sets of dimensions to get things in the right order
    L_matrix = []
    for i in range(H.shape[0]):
        for j in range(H.shape[0]):
            temp = []
            for k in range(H.shape[0]):
                for l in range(H.shape[0]):
                    temp.append(L[k][i][j][l])
            L_matrix.append(temp)
    L_matrix = sym.Matrix(L_matrix)

    return L_matrix

## 2 Level Case
We want to start with sysem with levels $|1\rangle$ and $|2\rangle$ to confirm that sympy is able to symbolically calculate the bloch equations.  We'll consider that we have a laser with frequency $\omega_L$ interacting with the atom and that the frequency associated with the transition is $\omega_{21} = \omega_2 - \omega_1$.

In [9]:
# Define the dimensionality here
dim = 2

In [21]:
# Define the symbols we need
freq_syms = generate_symbols("ω", [i+1 for i in range(dim)])
rabi_syms = [generate_symbols("Ω", [str(i+1) + "_" + str(i+2) for i in range(dim-1)]),
             generate_symbols("Ω", [str(i+2) + "_" + str(i+1) for i in range(dim-1)])]

In [11]:
# Define the Hamiltonian under the RWA and with zero detuning between the laser and transition
H = generate_H(dim, freq_syms, rabi_syms)

# Define the density matrix
ρ = generate_ρ(dim)

In [12]:
# Define damping rate
γ12 = sym.symbols("γ_1_2")

# Define jump operator
σ12 = generate_jump_ops(dim)[0]

In [13]:
# Get the master equation
dρ = generate_Liouvillian(ρ, H, [σ12], [γ12])
dρ

Matrix([
[                                 1.0*γ_1_2*(1.0 - 1.0*ρ_1_1) - 1.0*I*(1.0*Ω_1_2*ρ_2_1 - 1.0*Ω_2_1*ρ_1_2), -0.5*γ_1_2*ρ_1_2 - 1.0*I*(-1.0*Ω_1_2*ρ_1_1 + 1.0*Ω_1_2*(1.0 - 1.0*ρ_1_1) + 1.0*ρ_1_2*ω_1 - 1.0*ρ_1_2*ω_2)],
[-0.5*γ_1_2*ρ_2_1 - 1.0*I*(1.0*Ω_2_1*ρ_1_1 - 1.0*Ω_2_1*(1.0 - 1.0*ρ_1_1) - 1.0*ρ_2_1*ω_1 + 1.0*ρ_2_1*ω_2),                                   -γ_1_2*(2.0 - 2.0*ρ_1_1)/2 - 1.0*I*(-1.0*Ω_1_2*ρ_2_1 + 1.0*Ω_2_1*ρ_1_2)]])

We want to try working with the vectorized master equation to see if we can obtain the steady state solution numerically.

In [17]:
# Get the vectorized Liouvillian operator
vec_L = generate_vector_Liouvillian(H, [σ12], [γ12])
vec_L

Matrix([
[           0,                            1.0*I*Ω_2_1,                            -1.0*I*Ω_1_2,    1.0*γ_1_2],
[ 1.0*I*Ω_1_2, -0.5*γ_1_2 - 1.0*I*(1.0*ω_1 - 1.0*ω_2),                                       0, -1.0*I*Ω_1_2],
[-1.0*I*Ω_2_1,                                      0, -0.5*γ_1_2 - 1.0*I*(-1.0*ω_1 + 1.0*ω_2),  1.0*I*Ω_2_1],
[           0,                           -1.0*I*Ω_2_1,                             1.0*I*Ω_1_2,   -1.0*γ_1_2]])

In [15]:
# Check that the vectorized Liouvillian applied to the vectorized density matrix gives the same thing
dρ_vec = vec_L @ ρ.reshape(4, 1)
sym.simplify(dρ_vec - dρ.reshape(4, 1))

Matrix([
[0],
[0],
[0],
[0]])

In [24]:
# Now diagonalize the vectorized Liouvillian to find the 0 eigenvalue and the eigenvector corresponding to the steady state
vec_L_fun = sym.lambdify(freq_syms + rabi_syms[0] + rabi_syms[1] + [γ12], vec_L)

vec_L_val = vec_L_fun(50, 100, 2, 2, 2)
eigs = np.linalg.eig(vec_L_val)
eigs

EigResult(eigenvalues=array([-1.00317840e+00-5.01596818e+01j, -1.00317840e+00+5.01596818e+01j,
       -3.41646459e-15+1.48322006e-19j, -1.99364319e+00-2.15877561e-17j]), eigenvectors=array([[ 0.0398569 -7.93335949e-04j,  0.0398569 +7.93335949e-04j,
         0.99840828+0.00000000e+00j,  0.70485573+0.00000000e+00j],
       [-0.00159174+3.16323351e-05j,  0.99840826+0.00000000e+00j,
        -0.03985662+7.97132357e-04j, -0.0563662 -1.12015777e-03j],
       [ 0.99840826+0.00000000e+00j, -0.00159174-3.16323351e-05j,
        -0.03985662-7.97132357e-04j, -0.0563662 +1.12015777e-03j],
       [-0.0398569 +7.93335949e-04j, -0.0398569 -7.93335949e-04j,
         0.00159426+1.08472194e-17j, -0.70485573+7.66644809e-20j]]))

In [26]:
# Looks like the third eigenvalue is basically 0
eigs[0][2], eigs[1][:, 2]

((-3.416464590201822e-15+1.483220062922092e-19j),
 array([ 0.99840828+0.00000000e+00j, -0.03985662+7.97132357e-04j,
        -0.03985662-7.97132357e-04j,  0.00159426+1.08472194e-17j]))

In [27]:
# Confirm that applying the Liouvillian to this eigenvalue basically gives back 0
vec_L_val @ eigs[1][:, 2] 

array([ 3.64725611e-16+1.52655666e-16j,  2.33928500e-15+3.77475828e-15j,
        6.70209375e-15-5.67254577e-16j, -3.64725611e-16-1.52655666e-16j])

## 3 Level Case
Now we'll move on to considering a 3 level system in the same fashion.  In this case, the atom has levels $|1\rangle, |2\rangle, |3\rangle$ with frequencies $\omega_1, \omega_2, \omega_3$ respectively.  We consider having two lasers.  One laser incoherently drives the $|1\rangle$ and $|2\rangle$ transition with frequency $\omega_{L1}$ and the other incoherently drives the $|2\rangle$ and $|3\rangle$ transition with frequency $\omega_{L2}$.

In [28]:
# Define the dimensionality
dim = 3

In [29]:
# Define the symbols we need
freq_syms = generate_symbols("ω", [i+1 for i in range(dim)])
rabi_syms = [generate_symbols("Ω", [str(i+1) + "_" + str(i+2) for i in range(dim-1)]),
             generate_symbols("Ω", [str(i+2) + "_" + str(i+1) for i in range(dim-1)])]

In [31]:
# Define the Hamiltonian under the RWA and with zero detuning between the laser and transition
H = generate_H(dim, freq_syms, rabi_syms)

# Define the density matrix
ρ = generate_ρ(dim)

In [36]:
# Define damping rate
γ12, γ23 = sym.symbols("γ_1_2, γ_2_3")

# Define jump operator
σ = generate_jump_ops(dim)

In [37]:
# Get the master equation
dρ = generate_Liouvillian(ρ, H, σ, [γ12, γ23])
dρ

Matrix([
[                                                   1.0*γ_1_2*ρ_2_2 - 1.0*I*(1.0*Ω_1_2*ρ_2_1 - 1.0*Ω_2_1*ρ_1_2),                                            -0.5*γ_1_2*ρ_1_2 - 1.0*I*(-1.0*Ω_1_2*ρ_1_1 + 1.0*Ω_1_2*ρ_2_2 - 1.0*Ω_3_2*ρ_1_3 + 1.0*ρ_1_2*ω_1 - 1.0*ρ_1_2*ω_2),                                                              -0.5*γ_2_3*ρ_1_3 - 1.0*I*(1.0*Ω_1_2*ρ_2_3 - 1.0*Ω_2_3*ρ_1_2 + 1.0*ρ_1_3*ω_1 - 1.0*ρ_1_3*ω_3)],
[-0.5*γ_1_2*ρ_2_1 - 1.0*I*(1.0*Ω_2_1*ρ_1_1 - 1.0*Ω_2_1*ρ_2_2 + 1.0*Ω_2_3*ρ_3_1 - 1.0*ρ_2_1*ω_1 + 1.0*ρ_2_1*ω_2),               -1.0*γ_1_2*ρ_2_2 + 1.0*γ_2_3*(-1.0*ρ_1_1 - 1.0*ρ_2_2 + 1.0) - 1.0*I*(-1.0*Ω_1_2*ρ_2_1 + 1.0*Ω_2_1*ρ_1_2 + 1.0*Ω_2_3*ρ_3_2 - 1.0*Ω_3_2*ρ_2_3), -0.5*γ_1_2*ρ_2_3 - 0.5*γ_2_3*ρ_2_3 - 1.0*I*(1.0*Ω_2_1*ρ_1_3 - 1.0*Ω_2_3*ρ_2_2 + 1.0*Ω_2_3*(-1.0*ρ_1_1 - 1.0*ρ_2_2 + 1.0) + 1.0*ρ_2_3*ω_2 - 1.0*ρ_2_3*ω_3)],
[                 -0.5*γ_2_3*ρ_3_1 - 1.0*I*(-1.0*Ω_2_1*ρ_3_2 + 1.0*Ω_3_2*ρ_2_1 - 1.0*ρ_3_1*ω_1 + 1.0*ρ_3_1*ω_3), -0.5*γ_1_2*ρ_3_2 - 0.5*γ_2_3

Now do the vectorized case

In [39]:
# Get the vectorized Liouvillian operator
vec_L = generate_vector_Liouvillian(H, σ, [γ12, γ23])
vec_L

Matrix([
[           0,                            1.0*I*Ω_2_1,                                      0,                            -1.0*I*Ω_1_2,    1.0*γ_1_2,                                                  0,                                       0,                                                   0,            0],
[ 1.0*I*Ω_1_2, -0.5*γ_1_2 - 1.0*I*(1.0*ω_1 - 1.0*ω_2),                            1.0*I*Ω_3_2,                                       0, -1.0*I*Ω_1_2,                                                  0,                                       0,                                                   0,            0],
[           0,                            1.0*I*Ω_2_3, -0.5*γ_2_3 - 1.0*I*(1.0*ω_1 - 1.0*ω_3),                                       0,            0,                                       -1.0*I*Ω_1_2,                                       0,                                                   0,            0],
[-1.0*I*Ω_2_1,                                      0,    

In [40]:
# Check that the vectorized Liouvillian applied to the vectorized density matrix gives the same thing
dρ_vec = vec_L @ ρ.reshape(dim ** 2, 1)
sym.simplify(dρ_vec - dρ.reshape(dim ** 2, 1))

Matrix([
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0]])

In [41]:
# Now diagonalize the vectorized Liouvillian to find the 0 eigenvalue and the eigenvector corresponding to the steady state
vec_L_fun = sym.lambdify(freq_syms + rabi_syms[0] + rabi_syms[1] + [γ12, γ23], vec_L)

vec_L_val = vec_L_fun(50, 100, 150, 2, 4, 2, 4, 1, 2)
eigs = np.linalg.eig(vec_L_val)
eigs

EigResult(eigenvalues=array([-9.97671934e-01-1.00398317e+02j, -9.97671934e-01+1.00398317e+02j,
       -5.04731770e-01+4.98420423e+01j, -5.04731770e-01-4.98420423e+01j,
       -1.51167969e+00-5.05562802e+01j, -1.51167969e+00+5.05562802e+01j,
        2.82194937e-15-3.59125503e-18j, -9.90463567e-01+8.91732121e-16j,
       -1.98136964e+00-1.72050921e-15j]), eigenvectors=array([[ 1.57935544e-03-3.61937084e-07j,  1.57935544e-03+3.61937084e-07j,
         3.99964170e-02+3.95547024e-04j,  3.99964170e-02-3.95547024e-04j,
        -1.82110819e-04+1.73637904e-03j, -1.82110819e-04-1.73637904e-03j,
         9.98395076e-01+0.00000000e+00j,  7.04867910e-01+0.00000000e+00j,
        -4.04271343e-01+3.76505618e-16j],
       [-6.32758024e-05+2.27168170e-07j,  7.92350649e-02-7.81127579e-04j,
         9.95246876e-01+0.00000000e+00j, -1.60243629e-03+1.58056006e-05j,
         1.58510013e-03-6.86376519e-05j, -2.96097634e-03-4.34770598e-03j,
        -3.99952306e-02+4.02517869e-04j, -5.64738764e-02-5.54808088e-04

In [42]:
# The 7th eigenvalue looks like it's close to 0
eigs[0][6], eigs[1][:, 6]

((2.8219493716466905e-15-3.591255031270161e-18j),
 array([ 9.98395076e-01+0.00000000e+00j, -3.99952306e-02+4.02517869e-04j,
         1.59819557e-03-3.20695373e-05j, -3.99952306e-02-4.02517869e-04j,
         1.61007148e-03+1.58712152e-17j, -6.46480643e-05+6.56660439e-07j,
         1.59819557e-03+3.20695373e-05j, -6.46480643e-05-6.56660439e-07j,
         2.62664175e-06+1.37196227e-19j]))

In [43]:
# Confirm that applying the Liouvillian to this eigenvalue basically gives back 0
vec_L_val @ eigs[1][:, 6] 

array([ 4.82903648e-16+7.13823664e-17j,  6.90878019e-15+2.22044605e-16j,
       -1.56060061e-15+1.40720768e-14j,  5.23051654e-15+1.77635684e-15j,
       -1.21827224e-15-8.51028807e-16j, -7.40384732e-15-1.03002290e-14j,
        2.15097037e-14-6.38378239e-16j, -2.09300477e-15+1.09162218e-16j,
        7.35371982e-16+7.79646440e-16j])

## 4 Level Case

In [44]:
# Define the dimensionality
dim = 4

In [45]:
# Define the symbols we need
freq_syms = generate_symbols("ω", [i+1 for i in range(dim)])
rabi_syms = [generate_symbols("Ω", [str(i+1) + "_" + str(i+2) for i in range(dim-1)]),
             generate_symbols("Ω", [str(i+2) + "_" + str(i+1) for i in range(dim-1)])]

In [46]:
# Define the Hamiltonian under the RWA and with zero detuning between the laser and transition
H = generate_H(dim, freq_syms, rabi_syms)

# Define the density matrix
ρ = generate_ρ(dim)

In [54]:
# Define damping rate
γs = [sym.symbols("γ_" + str(i+1) + "_" + str(i+2)) for i in range(dim-1)]

# Define jump operator
σ = generate_jump_ops(dim)

In [55]:
# Get the master equation
dρ = generate_Liouvillian(ρ, H, σ, γs)
dρ

Matrix([
[                                                    1.0*γ_1_2*ρ_2_2 - 1.0*I*(1.0*Ω_1_2*ρ_2_1 - 1.0*Ω_2_1*ρ_1_2),                                     -0.5*γ_1_2*ρ_1_2 - 1.0*I*(-1.0*Ω_1_2*ρ_1_1 + 1.0*Ω_1_2*ρ_2_2 - 1.0*Ω_3_2*ρ_1_3 + 1.0*ρ_1_2*ω_1 - 1.0*ρ_1_2*ω_2),                                                         -0.5*γ_2_3*ρ_1_3 - 1.0*I*(1.0*Ω_1_2*ρ_2_3 - 1.0*Ω_2_3*ρ_1_2 - 1.0*Ω_4_3*ρ_1_4 + 1.0*ρ_1_3*ω_1 - 1.0*ρ_1_3*ω_3),                                                                          -0.5*γ_3_4*ρ_1_4 - 1.0*I*(1.0*Ω_1_2*ρ_2_4 - 1.0*Ω_3_4*ρ_1_3 + 1.0*ρ_1_4*ω_1 - 1.0*ρ_1_4*ω_4)],
[ -0.5*γ_1_2*ρ_2_1 - 1.0*I*(1.0*Ω_2_1*ρ_1_1 - 1.0*Ω_2_1*ρ_2_2 + 1.0*Ω_2_3*ρ_3_1 - 1.0*ρ_2_1*ω_1 + 1.0*ρ_2_1*ω_2),                                 -1.0*γ_1_2*ρ_2_2 + 1.0*γ_2_3*ρ_3_3 - 1.0*I*(-1.0*Ω_1_2*ρ_2_1 + 1.0*Ω_2_1*ρ_1_2 + 1.0*Ω_2_3*ρ_3_2 - 1.0*Ω_3_2*ρ_2_3),                     -0.5*γ_1_2*ρ_2_3 - 0.5*γ_2_3*ρ_2_3 - 1.0*I*(1.0*Ω_2_1*ρ_1_3 - 1.0*Ω_2_3*ρ_2_2 + 1.0*Ω_2_3*ρ_3_3 - 1.0*Ω_4_3*ρ

Now do the vectorized version

In [56]:
# Get the vectorized Liouvillian operator
vec_L = generate_vector_Liouvillian(H, σ, γs)
vec_L

Matrix([
[           0,                            1.0*I*Ω_2_1,                                      0,                                      0,                            -1.0*I*Ω_1_2,    1.0*γ_1_2,                                                  0,                                                  0,                                       0,                                                   0,            0,                                                  0,                                       0,                                                   0,                                                   0,            0],
[ 1.0*I*Ω_1_2, -0.5*γ_1_2 - 1.0*I*(1.0*ω_1 - 1.0*ω_2),                            1.0*I*Ω_3_2,                                      0,                                       0, -1.0*I*Ω_1_2,                                                  0,                                                  0,                                       0,                                         

In [57]:
# Check that the vectorized Liouvillian applied to the vectorized density matrix gives the same thing
dρ_vec = vec_L @ ρ.reshape(dim ** 2, 1)
sym.simplify(dρ_vec - dρ.reshape(dim ** 2, 1))

Matrix([
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0]])

In [58]:
# Now diagonalize the vectorized Liouvillian to find the 0 eigenvalue and the eigenvector corresponding to the steady state
vec_L_fun = sym.lambdify(freq_syms + rabi_syms[0] + rabi_syms[1] + γs, vec_L)

vec_L_val = vec_L_fun(50, 100, 150, 200, 2, 4, 6, 2, 4, 6, 1, 2, 3)
eigs = np.linalg.eig(vec_L_val)
eigs[0]

array([-1.49383732e+00+1.50792174e+02j, -1.49383732e+00-1.50792174e+02j,
       -1.00457096e+00+9.96885382e+01j, -1.00457096e+00-9.96885382e+01j,
       -1.99566603e+00+1.00952402e+02j, -1.99566603e+00-1.00952402e+02j,
       -5.04799330e-01+4.98397695e+01j, -1.51860560e+00+4.98487746e+01j,
       -5.04799330e-01-4.98397695e+01j, -1.51860560e+00-4.98487746e+01j,
       -2.53764792e+00-5.11036965e+01j, -2.53764792e+00+5.11036965e+01j,
        1.83024625e-15+6.66242850e-18j, -9.90276036e-01-5.76031131e-15j,
       -1.95268038e+00+7.35834806e-15j, -2.94678927e+00-2.45545569e-15j])

In [60]:
# Looks like the 13th eigenvalue is pretty close to 0
# Confirm that applying the Liouvillian to this eigenvector basically gives back 0
vec_L_val @ eigs[1][:, 12] 

array([-1.02413737e-15-8.32667268e-17j, -1.72847305e-14+2.22044605e-15j,
       -6.81746326e-15+7.91033905e-15j,  2.58623147e-14+6.75050293e-14j,
       -1.04922383e-14-5.10485751e-15j,  1.24151991e-15-6.80011603e-16j,
       -1.68245371e-15-8.38652064e-15j,  3.50216360e-15-5.10084596e-15j,
       -1.41008624e-15-8.07622198e-15j,  4.25318960e-15-1.66923766e-15j,
       -2.44237715e-16+5.42480557e-16j,  2.34455924e-15-3.73174595e-15j,
        1.62991692e-14-2.63642842e-14j, -8.14822148e-15-2.14062166e-15j,
       -2.78446922e-15+2.42691372e-15j,  2.68212313e-17+2.18092984e-16j])

Note that we still have to incorporate the cavity 